<a href="https://colab.research.google.com/github/RoboMaroof/LLM-Applications-Building-Blocks/blob/main/LLM__Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs and Imports

In [1]:
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.2 MB/s eta 0:00:00


In [6]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoModel,
    AutoTokenizer,
    AutoConfig,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

# Hugging Face Login

In [5]:
from huggingface_hub import notebook_login
notebook_login()

# Load Model

In [3]:
model_name = "google/gemma-2b-it"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
config = AutoConfig.from_pretrained(model_name)
print(config)

GemmaConfig {
  "_name_or_path": "google/gemma-2b-it",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 16384,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.0",
  "use_cache": true,
  "vocab_size": 256000
}



In [ ]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRM

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter: {name}, dtype: {param.dtype}")

Parameter: model.embed_tokens.weight, dtype: torch.float32
Parameter: model.layers.0.self_attn.q_proj.weight, dtype: torch.float32
Parameter: model.layers.0.self_attn.k_proj.weight, dtype: torch.float32
Parameter: model.layers.0.self_attn.v_proj.weight, dtype: torch.float32
Parameter: model.layers.0.self_attn.o_proj.weight, dtype: torch.float32
Parameter: model.layers.0.mlp.gate_proj.weight, dtype: torch.float32
Parameter: model.layers.0.mlp.up_proj.weight, dtype: torch.float32
Parameter: model.layers.0.mlp.down_proj.weight, dtype: torch.float32
Parameter: model.layers.0.input_layernorm.weight, dtype: torch.float32
Parameter: model.layers.0.post_attention_layernorm.weight, dtype: torch.float32
Parameter: model.layers.1.self_attn.q_proj.weight, dtype: torch.float32
Parameter: model.layers.1.self_attn.k_proj.weight, dtype: torch.float32
Parameter: model.layers.1.self_attn.v_proj.weight, dtype: torch.float32
Parameter: model.layers.1.self_attn.o_proj.weight, dtype: torch.float32
Parameter

# Casting

In [ ]:
# Cast model parameters to lower precision bfloat16 (if supported by GPU)
model = model.to(dtype=torch.bfloat16)

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter: {name}, dtype: {param.dtype}")

# Load model in bfloat16 (if supported by model and GPU)

In [ ]:
model = AutoModel.from_pretrained(model_name, torch_dtype=torch.bfloat16)

In [13]:
for name, param in model.named_parameters():
    print(f"Parameter: {name}, Data Type: {param.dtype}")

In [ ]:
print(tokenizer)

# Additional loading parameters

## Common Options for `from_pretrained`

### `pretrained_model_name_or_path`
Specifies the name of the pre-trained model or the path to the local directory containing the model weights and configuration files.
- **Example:** `'bert-base-uncased'` or `'/path/to/local/model'`.

### `config`
Accepts a configuration object (`PretrainedConfig` subclass) that overrides the default model configuration.
- **Example:** `config=AutoConfig.from_pretrained('bert-base-uncased')`.

### `state_dict`
Provides a state dictionary (a Python dictionary object) with custom model weights.
- **Example:** `state_dict=custom_state_dict`.

### `cache_dir`
Specifies the directory where the downloaded model weights and configuration files will be cached.
- **Example:** `cache_dir='./cache'`.

### `force_download`
Forces the download of the model weights and configuration files, even if they are already cached.
- **Example:** `force_download=True`.

### `resume_download`
Resumes downloading a model that was partially downloaded.
- **Example:** `resume_download=True`.

### `proxies`
A dictionary of proxy servers to use by protocol or endpoint.
- **Example:** `proxies={'http': 'http://10.10.1.10:3128', 'https': 'http://10.10.1.10:1080'}`.

### `use_auth_token`
The token to use as HTTP bearer authorization for remote files.
- **Example:** `use_auth_token='your_token_here'`.

### `revision`
Specifies the model version to use (can be a branch name, tag name, or commit id).
- **Example:** `revision='main'` or `revision='v1.2.3'`.

### `subfolder`
Loads the model from a subfolder in the repository.
- **Example:** `subfolder='my_model'`.

### `mirror`
Specifies the base URL of the mirror to use to download the model weights and configuration files.
- **Example:** `mirror='https://mirror.s3.amazonaws.com'`.

## Precision and Device Management

### `torch_dtype`
Specifies the desired data type for the model parameters. Useful for reducing memory usage or leveraging specific hardware features.
- **Example:** `torch_dtype=torch.bfloat16` or `torch_dtype=torch.float16`.

### `device_map`
A dictionary specifying which device to place each layer of the model on. Useful for model parallelism.
- **Example:** `device_map={'layer.0': 'cuda:0', 'layer.1': 'cuda:1'}`.

### `load_in_8bit`
Loads the model in 8-bit precision using the bitsandbytes library, which can significantly reduce memory usage.
- **Example:** `load_in_8bit=True`.

### `quantization_config`
A configuration object for applying quantization-aware training or post-training quantization.
- **Example:** `quantization_config=my_quantization_config`.

### `low_cpu_mem_usage`
Reduces the CPU memory usage by loading the model in a more memory-efficient way, particularly useful when working with very large models.
- **Example:** `low_cpu_mem_usage=True`.


In [ ]:
# Example
model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,         # Set precision to bfloat16
    cache_dir='./cache',                # Set cache directory
    force_download=True,                # Force re-download of the model
    low_cpu_mem_usage=True              # Reduce CPU memory usage
)